https://maap-ops-dataset.s3.amazonaws.com/maap-users/alexdevseed/gee/TTE_DATA_v201902v2-class-1000.zip.tif
s3://maap-ops-dataset/maap-users/alexdevseed/gee/TTE_DATA_v201902v2-class-1000.zip.tif

In [ ]:
#for zip in glob
# unzip renaming tif inside to match zip tilename
# better idea rio cogeo /vsizip/nameof.zip nameof.tif
!cd /projects/tmp/gee
!less todo.txt | parallel --bar -j 10 ./maketif.sh {}

In [ ]:
!pip install cogeo-mosaic

In [14]:
from glob import glob
import os
import json
from typing import Dict

from cogeo_mosaic.mosaic import MosaicJSON
from cogeo_mosaic.backends import MosaicBackend

In [6]:
def local_to_s3(url):
    return url.replace("/projects/maap-users", "s3://maap-ops-dataset/maap-users")
    
def s3_to_https(url):
    parts = url.split("/")
    return os.path.join('https://',f'{parts[2]}.s3.amazonaws.com', *parts[3:])

In [18]:
#glob find the tifs
tif_dir = "/projects/maap-users/alexdevseed/gee/"

local_tiles = glob(os.path.join(tif_dir,"*class*.zip.tif"), recursive=False)
tiles = [local_to_s3(tile) for tile in local_tiles]

In [20]:
len(tiles)

2304

In [ ]:

#tiles = [
#    's3://maap-ops-dataset/maap-users/alexdevseed/landsat8/viz/Landsat8_30542_comp_cog_2015-2020_dps.tif',
#    's3://maap-ops-dataset/maap-users/alexdevseed/landsat8/viz/Landsat8_30543_comp_cog_2015-2020_dps.tif',
#    's3://maap-ops-dataset/maap-users/alexdevseed/landsat8/viz/Landsat8_30822_comp_cog_2015-2020_dps.tif',
#    's3://maap-ops-dataset/maap-users/alexdevseed/landsat8/viz/Landsat8_30823_comp_cog_2015-2020_dps.tif',
#]


mosaicdata = MosaicJSON.from_urls(tiles, minzoom=8, maxzoom=16)

In [22]:
mosaic_json = "/projects/maap-users/alexdevseed/gee/forest_classificiation.json"
with MosaicBackend(mosaic_json, mosaic_def=mosaicdata) as mosaic:
    mosaic.write(overwrite=True)

## From Features

In [15]:
def get_accessor(feature: Dict):
    """Return specific feature identifier."""
    return feature["properties"]["s3"]

In [25]:
with open("/projects/maap-users/alexdevseed/gee/tile_index.geojson") as f:
    geojson = json.load(f)

In [26]:
mosaicdata = MosaicJSON.from_features(geojson.get('features'), minzoom=8, maxzoom=16, accessor=get_accessor)

In [28]:
mosaic_json = "/projects/maap-users/alexdevseed/gee/forest_classificiation.json"
with MosaicBackend(mosaic_json, mosaic_def=mosaicdata) as mosaic:
    mosaic.write(overwrite=True)

In [27]:
geojson.get('features')[0]

{'type': 'Feature',
 'properties': {'Name': 'W180N45',
  'box': '-180,45,-179,46',
  'id': 0,
  'zip': '/projects/tmp/gee/TTE_DATA_v201902v2-class-0.zip',
  'tif': '/projects/maap-users/alexdevseed/gee/TTE_DATA_v201902v2-class-0.zip.tif',
  's3': 's3://maap-ops-dataset/maap-users/alexdevseed/gee/TTE_DATA_v201902v2-class-0.zip.tif'},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-179.0, 45.0],
    [-179.0, 46.0],
    [-180.0, 46.0],
    [-180.0, 45.0],
    [-179.0, 45.0]]]}}

## Visual Check

In [5]:
import ipycmc

In [7]:
wmts_url = "https://baxpil3vd6.execute-api.us-east-1.amazonaws.com/mosaicjson/WMTSCapabilities.xml"
params = [
    "tile_format=png",
    "tile_scale=1",
    "pixel_selection=first",
    "TileMatrixSetId=WebMercatorQuad",
    "url=s3%3A%2F%2Fmaap-ops-dataset%2Fmaap-users%2Falexdevseed%2Fgee%2Fforest_classificiation.json",
    "bidx=1",
    "resampling_method=nearest",
    "return_mask=true",
    "colormap_name=viridis"
]
#TODO url encode so the params can be human readable
wmts_call = "?".join([wmts_url, "&".join([*params])])
wmts_call

'https://baxpil3vd6.execute-api.us-east-1.amazonaws.com/mosaicjson/WMTSCapabilities.xml?tile_format=png&tile_scale=1&pixel_selection=first&TileMatrixSetId=WebMercatorQuad&url=s3%3A%2F%2Fmaap-ops-dataset%2Fmaap-users%2Falexdevseed%2Fgee%2Fforest_classificiation.json&bidx=1&resampling_method=nearest&return_mask=true&colormap_name=viridis'

In [ ]:
w = ipycmc.MapCMC()
w

In [ ]:
# this layer is crashing CMC
#w.load_layer_config(wmts_call
#                    , "wmts/xml")